In [ ]:
import os
import yaml
import optuna
import json
import numpy as np

from src.save_load import buildAuditMetadata, saveAudit
from src.train_models import trainTargetModel
from src.visualize_model import VisualizeModel
from src.utils import print_yaml

In [ ]:
# ------------------- #
#   Load Audit yaml   #
# ------------------- #
config = None
with open("./audit.yaml") as file:
    audit_config = yaml.safe_load(file)

print("-------------- Audit config --------------")
print_yaml(audit_config['audit'])
print_yaml(audit_config['target'])

# -------------------------------- #
#  Load target model and metadata  #
# -------------------------------- #
target_folder = "resnet18-abeb4ef939"
target_path = os.path.join("target", target_folder)

# Metadata and target .pkl path for audit config
target_model_path = os.path.join(target_path, "target_model.pkl")
target_metadata_path = os.path.join(target_path, "target_metadata.pkl")

# Update the audit config
audit_config['target']['target_folder'] = target_path

print("\n-------------- Updated audit config --------------")
print_yaml(audit_config['audit'])
print_yaml(audit_config['target'])

# ------------------- #
#   Load Train yaml   #
# ------------------- #
config = None
with open("./train.yaml") as file:
    train_config = yaml.safe_load(file)
    
print("\n-------------- Train config --------------")
print_yaml(train_config['train'])

# Target Metadata .json
metadata_path = os.path.join(target_path, "target_metadata.pkl")
metadata_path = os.path.join(target_path, "metadata.json")
with open(metadata_path, "r") as f:
    metadata = json.load(f)

# Update the train config with target metadata (learning rate, batch size, etc)
train_config['train']['epochs'] = metadata['train']['epochs']
train_config['train']['batch_size'] = metadata['train']['batch_size']
train_config['train']['learning_rate'] = metadata['train']['learning_rate']
train_config['train']['momentum'] = metadata['train']['momentum']

train_config['run']['log_dir'] = target_path

print("\n-------------- Updated train config --------------")
print_yaml(train_config['train'])

In [ ]:
from LeakPro.leakpro.schemas import LeakProConfig
from LeakPro.leakpro.attacks.mia_attacks.lira import AttackLiRA
from LeakPro.leakpro.attacks.utils.shadow_model_handler import ShadowModelHandler
from LeakPro.leakpro.input_handler.mia_handler import MIAHandler
from src.cifar_handler import CifarInputHandler
# ----------------- #
#   Setup LeakPro   #
# ----------------- #

# Intizializing
leakpro_configs = LeakProConfig(**audit_config)
print("-------- LeakPro Configs --------")
print_yaml(leakpro_configs)

handler = MIAHandler(leakpro_configs, CifarInputHandler)

configs = handler.configs.audit.attack_list[0]
print("-------- Attack Configs --------")
print_yaml(configs)

attack = AttackLiRA(handler=handler, configs=configs)

In [ ]:
# --------------------------- #
#   Train the Shadow models   #
# --------------------------- #

#Set number of shadow models to train
num_shadow_models = configs["num_shadow_models"]
#Set online flag
online = configs["online"]

attack_data_indices = attack.sample_indices_from_population(include_train_indices = online,
                                                        include_test_indices = online)

training_data_fraction = attack.training_data_fraction

smh = ShadowModelHandler(handler)
smh.epochs = train_config["train"]["epochs"]
smh.batch_size = train_config['train']['batch_size']
smh.learning_rate = train_config['train']['learning_rate']
smh.momentum = train_config['train']['momentum']

shadow_model_indices = smh.create_shadow_models(num_models = num_shadow_models,
                                                                shadow_population =  attack_data_indices,
                                                                training_fraction = training_data_fraction,
                                                                online = online,
                                                                #verbose = False,
                                                                #incremental = INCREMENTAL, 
                                                                #shuffle_shift = SHUFFLE_SHIFT
                                                                       )

In [ ]:
# -------------------------------- #
#   Load Shadow Models   #
# -------------------------------- #
#Get shadow models
shadow_models, _ = smh.get_shadow_models(shadow_model_indices)

In [ ]:
# -------------------------------- #
#   Extract Shadow model Signals   #
# -------------------------------- #

# Get the audit dataset from the attack
audit_dataset = attack.audit_dataset
true_labels = handler.get_labels(audit_dataset["data"])
print(f"\nTrue labels fetched: {true_labels[:10]}")

# Fetch and rescale shadow model logits
shadow_models_logits = []
for indx in shadow_model_indices:
    shadow_models_logits.append(smh.load_logits(indx=indx))
rescaled_sm_logits = np.array([attack.rescale_logits(x, true_labels) for x in shadow_models_logits])

print("\n--------- Shadow models logits extracted ---------")
print(rescaled_sm_logits[0])
print(f"Shadow model logits shape: {rescaled_sm_logits.shape}")

# Extract target model logits

target_logits = smh.load_logits(name="target")
rescaled_target_logits = attack.rescale_logits(target_logits, true_labels)

print("\n--------- target model logits extracted ---------")
print(rescaled_target_logits[:10])

In [ ]:
# Get the in indices mask for shadow models
in_indices_masks = ShadowModelHandler(handler).get_in_indices_mask(shadow_model_indices, audit_dataset["data"])#.astype(int)

audit_data_indices = audit_dataset["data"]

In [ ]:
# ------------------------------ #
#   Save logits & indices mask   #
# ------------------------------ #
metadata = buildAuditMetadata(train_config, audit_config)

hash_id, save_dir = saveAudit(metadata, rescaled_target_logits, rescaled_sm_logits, in_indices_masks, audit_data_indices)